### Model Engenering Titanic
Use 3 Model
> Gradient Biisting(LGBM) 

>RandomTreee

>PyTorch NN

In [22]:
import optuna
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

 Download data

In [15]:
data = pd.read_csv('Titanic_EDA.csv')
data.columns

Index(['Parch_0', 'Parch_1', 'Parch_2', 'Parch_3', 'SibSp_0', 'SibSp_1',
       'SibSp_2', 'SibSp_3', 'Fare', 'Survived', 'Pclass', 'Age', 'Sex',
       'Embar_C', 'Embar_Q', 'Embar_S', 'Size Family'],
      dtype='object')

Prepare model to Train Test part

In [16]:
X_train,X_test,Y_train,Y_test = train_test_split(data.drop(columns = ['Survived']),data['Survived'],train_size=0.8,random_state=42)

Let's test several Model


In [17]:
def pred(model,X_test=X_test,Y_test=Y_test):
    print(f1_score(model.predict(X_test),Y_test))
    print(accuracy_score(model.predict(X_test),Y_test))



Use Optuna for find the best hiperparametrs

### LGBM

In [20]:
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'num_leaves': trial.suggest_int('num_leaves', 10, 50),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50)
    }
    model = LGBMClassifier(**params, random_state=42,verbose=-1)
    score = cross_val_score(model, X_train, Y_train, cv=5, scoring='f1').mean()
    
    return score

In [ ]:
study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials=50)
print("Лучшие параметры:", study.best_params)
print("Лучший скор:", study.best_value)

[I 2025-03-18 23:00:35,258] A new study created in memory with name: no-name-4f20a74a-d5e4-4f8b-8ed8-271bbd2d8a7f
[I 2025-03-18 23:00:35,921] Trial 0 finished with value: 0.7233783113158725 and parameters: {'learning_rate': 0.22836566732654862, 'n_estimators': 834, 'num_leaves': 31, 'max_depth': 6, 'min_child_samples': 25}. Best is trial 0 with value: 0.7233783113158725.
[I 2025-03-18 23:00:36,173] Trial 1 finished with value: 0.7506030708153807 and parameters: {'learning_rate': 0.10773962101967734, 'n_estimators': 491, 'num_leaves': 10, 'max_depth': 5, 'min_child_samples': 20}. Best is trial 1 with value: 0.7506030708153807.
[I 2025-03-18 23:00:36,359] Trial 2 finished with value: 0.7392120063070136 and parameters: {'learning_rate': 0.01999132259489976, 'n_estimators': 341, 'num_leaves': 34, 'max_depth': 5, 'min_child_samples': 45}. Best is trial 1 with value: 0.7506030708153807.
[I 2025-03-18 23:00:36,722] Trial 3 finished with value: 0.7451891031186995 and parameters: {'learning_rat

Лучшие параметры: {'learning_rate': 0.09910831490174508, 'n_estimators': 224, 'num_leaves': 40, 'max_depth': 5, 'min_child_samples': 26}
Лучший скор: 0.7711779448621554


In [9]:
model = LGBMClassifier(**study.best_params,verbose = -1)
model.fit(X_train,Y_train)
pred(model)

0.7299270072992701
0.7897727272727273


### Catboost

In [ ]:
from catboost import CatBoostClassifier
def objectiveCat(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'iterations': trial.suggest_int('iterations', 100, 1000),  
        'depth': trial.suggest_int('depth', 3, 10),  
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),  
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 0.1, 10)
    }
    
  
    model = CatBoostClassifier(**params, random_seed=42, silent=True)
    score = cross_val_score(model, X_train, Y_train, cv=5, scoring='accuracy').mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objectiveCat, n_trials=50)

print("Лучшие параметры:", study.best_params)
print("Лучший f1_score:", study.best_value)



[I 2025-03-18 23:04:55,780] A new study created in memory with name: no-name-5fff8c95-4d6c-4c1b-b5d4-c0648bb6e85c
[I 2025-03-18 23:05:04,385] Trial 0 finished with value: 0.8196048632218845 and parameters: {'learning_rate': 0.10438428280658842, 'iterations': 974, 'depth': 7, 'l2_leaf_reg': 2.2883137917175427, 'border_count': 193, 'random_strength': 6.490367498944651}. Best is trial 0 with value: 0.8196048632218845.
[I 2025-03-18 23:05:10,244] Trial 1 finished with value: 0.8125025329280648 and parameters: {'learning_rate': 0.13586619052220078, 'iterations': 703, 'depth': 7, 'l2_leaf_reg': 5.860623185744146, 'border_count': 141, 'random_strength': 4.174255403233249}. Best is trial 0 with value: 0.8196048632218845.
[I 2025-03-18 23:05:15,771] Trial 2 finished with value: 0.8153292806484295 and parameters: {'learning_rate': 0.17239086596121317, 'iterations': 644, 'depth': 7, 'l2_leaf_reg': 4.113490054547803, 'border_count': 159, 'random_strength': 9.187678363066901}. Best is trial 0 with 

Лучшие параметры: {'learning_rate': 0.2655927058204658, 'iterations': 106, 'depth': 9, 'l2_leaf_reg': 8.01500774354962, 'border_count': 243, 'random_strength': 3.2696265342516075}
Лучший f1_score: 0.8352077001013172


In [13]:

best_params = study.best_params
final_model = CatBoostClassifier(**best_params, random_seed=42, silent=True)
final_model.fit(X_train, Y_train)
pred(final_model)

0.7384615384615385
0.8068181818181818


RandomTreeClassifer

In [ ]:
def objectiveRandom(trial):
    params = {
        
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),

        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10)
    }
    model = RandomForestClassifier(**params, random_state=42)
    score = cross_val_score(model, X_train, Y_train, cv=5, scoring='f1').mean()
    
    return score
    
study = optuna.create_study(direction='maximize')
study.optimize(objectiveRandom, n_trials=50)

print("Лучшие параметры:", study.best_params)
print("Лучший f1_score:", study.best_value)


[I 2025-03-19 09:18:23,588] A new study created in memory with name: no-name-57d3ed0c-a628-49e9-a4c5-15afab7b3d94
[I 2025-03-19 09:18:25,333] Trial 0 finished with value: 0.7478798371175979 and parameters: {'n_estimators': 308, 'max_depth': 7, 'min_sample_split': 6}. Best is trial 0 with value: 0.7478798371175979.
[I 2025-03-19 09:18:26,161] Trial 1 finished with value: 0.7523201099102043 and parameters: {'n_estimators': 140, 'max_depth': 12, 'min_sample_split': 8}. Best is trial 1 with value: 0.7523201099102043.
[I 2025-03-19 09:18:28,208] Trial 2 finished with value: 0.7566852313688128 and parameters: {'n_estimators': 354, 'max_depth': 15, 'min_sample_split': 9}. Best is trial 2 with value: 0.7566852313688128.
[I 2025-03-19 09:18:29,877] Trial 3 finished with value: 0.733478121749651 and parameters: {'n_estimators': 317, 'max_depth': 4, 'min_sample_split': 10}. Best is trial 2 with value: 0.7566852313688128.
[I 2025-03-19 09:18:34,749] Trial 4 finished with value: 0.7423609388939112 

Лучшие параметры: {'n_estimators': 106, 'max_depth': 30, 'min_sample_split': 6}
Лучший f1_score: 0.76619249598628


In [31]:
model = RandomForestClassifier(n_estimators=106,max_depth=30,min_samples_split=6,random_state=42)
model.fit(X_train,Y_train)
pred(model)

0.6950354609929078
0.7584269662921348


### Torch Model

In [49]:
import torch
from torch import nn
class StandartLayer(nn.Module):
    def __init__(self,in_feat,out_feat,prob = 0.5):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_features= in_feat,out_features=out_feat),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=out_feat),
            nn.Dropout1d(p = prob)
        )
    def forward(self,x):
        return self.fc(x)

class TitanicNN(nn.Module):
    def __init__(self,layers,dropout):
        super().__init__()
        self.fc = nn.Sequential(
            *[StandartLayer(layers[i-1] ,layers[i],dropout[i-1]) for i in range(1,len(layers))]
        )
        self.res = nn.Linear(layers[-1],1)
        self.sig = nn.Sigmoid()
    def forward(self,x):
        x = self.fc(x)
        x = self.res(x)
        return self.sig(x)

X_train.shape

(711, 16)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim


x_train = torch.tensor(X_train.values,dtype = torch.float32)  
y_train = torch.tensor(Y_train.values,dtype= torch.float32)  

dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

input_size = 17
layers = [16,64, 32, 16]
dropout = [0.5, 0.3, 0.1]
model = TitanicNN( layers, dropout)


optimizer = optim.Adam(model.parameters(), lr=0.1)
criterion = nn.BCELoss()  


num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train() 
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)  
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item()
        predicted = (outputs.squeeze() >= 0.5).float() 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

print("Обучение завершено!")

Epoch 1/50, Loss: 0.7388, Accuracy: 58.37%
Epoch 2/50, Loss: 0.6667, Accuracy: 62.17%
Epoch 3/50, Loss: 0.6773, Accuracy: 61.04%
Epoch 4/50, Loss: 0.6736, Accuracy: 62.45%
Epoch 5/50, Loss: 0.6628, Accuracy: 62.45%
Epoch 6/50, Loss: 0.6735, Accuracy: 61.74%
Epoch 7/50, Loss: 0.6508, Accuracy: 62.03%
Epoch 8/50, Loss: 0.6833, Accuracy: 60.76%
Epoch 9/50, Loss: 0.6661, Accuracy: 62.31%
Epoch 10/50, Loss: 0.6486, Accuracy: 64.42%
Epoch 11/50, Loss: 0.6337, Accuracy: 66.10%
Epoch 12/50, Loss: 0.6620, Accuracy: 63.71%
Epoch 13/50, Loss: 0.6310, Accuracy: 65.96%
Epoch 14/50, Loss: 0.6800, Accuracy: 59.49%
Epoch 15/50, Loss: 0.6663, Accuracy: 62.17%
Epoch 16/50, Loss: 0.6531, Accuracy: 63.85%
Epoch 17/50, Loss: 0.6479, Accuracy: 65.54%
Epoch 18/50, Loss: 0.6278, Accuracy: 63.01%
Epoch 19/50, Loss: 0.6428, Accuracy: 63.43%
Epoch 20/50, Loss: 0.6072, Accuracy: 66.81%
Epoch 21/50, Loss: 0.6024, Accuracy: 68.50%
Epoch 22/50, Loss: 0.5954, Accuracy: 69.48%
Epoch 23/50, Loss: 0.6189, Accuracy: 67.6